### Import google drive

In [1]:
!pip install -U -q PyDrive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


### Import librares

In [2]:
import pandas as pd
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import *
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tqdm import tqdm

!pip install -q pyyaml h5py
import tensorflow as tf
from keras import backend as K
import os
# import keras
from keras import Sequential
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.layers import Conv1D, MaxPool1D, Dense, Dropout, Flatten, PReLU
from keras.optimizers import Adam
from keras.losses import MAPE
from keras.losses import MAE

def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


In [0]:
PATH = '/content/drive/My Drive/Colab Notebooks/mipt/'
target_col = 'RTA'
models_score = {}

### Load data

In [0]:
train = pd.read_csv(PATH + 'train_clean.csv', index_col=0)
valid = pd.read_csv(PATH + 'valid_clean.csv', index_col=0)
test = pd.read_csv(PATH + 'test_clean.csv', index_col=0)

In [5]:
train

,ETA,RTA,latitude,del_latitude,longitude,del_longitude,EDA,route_steps,route_dist,route_dist_sum,haversine_dist,manhattan_dist,havers_pickup_center,havers_dropoff_center,angle,over_range,ready_time,collcted_time,ready_collected_dif,hour,day_of_week,hour_of_week,month,day_of_year,day_of_month,holiday,pickup_cluster,dropoff_cluster,pickup_pca0,pickup_pca1,dropoff_pca0,dropoff_pca1,location_1078,location_22390,location_22430,location_22438
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,226.0,188.0,0.894129,0.872610,-0.198997,-0.311704,1.0,29,1.092981,0.039035,0.475936,0.522207,3.799501,3.599801,-95.879608,2.101121,2.450000,3.000000,0.550000,19,2,67,2,43,12,0,71,71,0.005226,-0.718199,-0.129636,-0.025526,1,0,0,0
1,718.0,725.0,-0.000943,0.735455,-0.255365,-0.521256,5.0,78,4.924563,0.063955,2.999397,3.832401,1.252477,2.958992,-19.603279,1.667002,6.683333,7.333333,0.650000,19,2,67,2,43,12,0,75,71,-0.285903,-0.104252,-0.395904,-0.025087,1,0,0,0
2,612.0,764.0,-0.131512,0.707553,1.487592,0.586587,5.0,79,5.139101,0.065886,4.458880,6.305716,7.169744,4.963267,-43.843021,1.121358,2.616667,2.683333,0.066667,19,2,67,2,43,12,0,14,73,1.630580,0.704824,0.825433,-0.003309,1,0,0,0
3,1560.0,1412.0,-1.221150,0.846165,0.795723,-0.912909,13.0,164,12.724716,0.078066,9.853176,13.803791,6.591928,3.547872,-36.946603,1.319372,6.266667,6.416667,0.150000,19,2,67,2,43,12,0,55,36,0.578971,1.196151,-0.802724,-0.036014,1,0,0,0
4,1528.0,893.0,0.786832,-0.267775,-1.404638,-0.012411,9.0,130,9.052936,0.070178,6.107029,8.606808,4.087524,2.171229,130.378310,1.473712,3.783333,4.683333,0.900000,19,2,67,2,43,12,0,15,22,-1.370801,-1.137194,-0.080026,0.016613,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
831219,1276.0,1194.0,-1.695371,0.128733,0.714213,0.021534,7.0,104,6.584588,0.063928,5.596670,7.500126,6.360555,0.829118,-26.316080,1.250744,6.700000,10.233333,3.533333,12,2,60,2,36,5,0,92,31,0.441416,2.074180,0.038427,0.005705,0,0,0,1
831220,2079.0,1542.0,-1.841886,0.414647,-0.332383,0.952439,13.0,165,13.142777,0.080139,7.799848,10.928429,6.370874,3.447571,37.020715,1.666699,5.000000,5.433333,0.433333,12,2,60,2,36,5,0,46,94,-0.572725,1.372705,0.961334,0.018681,0,0,0,1
831221,268.0,247.0,-0.155807,-0.090854,-1.126549,-1.358342,1.0,24,1.368614,0.059505,0.958543,1.110656,4.406609,5.247513,-79.977938,1.043250,3.616667,5.300000,1.683333,12,2,60,2,36,5,0,59,59,-1.091350,-0.756752,-1.301031,-0.018398,0,0,0,1


### Drop columns

In [6]:
print(train.columns[:])
columns_skip = []

Index(['ETA', 'RTA', 'latitude', 'del_latitude', 'longitude', 'del_longitude',
       'EDA', 'route_steps', 'route_dist', 'route_dist_sum', 'haversine_dist',
       'manhattan_dist', 'havers_pickup_center', 'havers_dropoff_center',
       'angle', 'over_range', 'ready_time', 'collcted_time',
       'ready_collected_dif', 'hour', 'day_of_week', 'hour_of_week', 'month',
       'day_of_year', 'day_of_month', 'holiday', 'pickup_cluster',
       'dropoff_cluster', 'pickup_pca0', 'pickup_pca1', 'dropoff_pca0',
       'dropoff_pca1', 'location_1078', 'location_22390', 'location_22430',
       'location_22438'],
      dtype='object')


In [0]:
if columns_skip:
    train = train[columns_skip]
    valid = valid[columns_skip]
    test = test[columns_skip]

## Prepare data

### One Hot Encoding

In [8]:
print(train.columns[:])
columns_encode = ['hour', 'day_of_week', 'month', 'pickup_cluster', 'dropoff_cluster', 'hour_of_week']

Index(['ETA', 'RTA', 'latitude', 'del_latitude', 'longitude', 'del_longitude',
       'EDA', 'route_steps', 'route_dist', 'route_dist_sum', 'haversine_dist',
       'manhattan_dist', 'havers_pickup_center', 'havers_dropoff_center',
       'angle', 'over_range', 'ready_time', 'collcted_time',
       'ready_collected_dif', 'hour', 'day_of_week', 'hour_of_week', 'month',
       'day_of_year', 'day_of_month', 'holiday', 'pickup_cluster',
       'dropoff_cluster', 'pickup_pca0', 'pickup_pca1', 'dropoff_pca0',
       'dropoff_pca1', 'location_1078', 'location_22390', 'location_22430',
       'location_22438'],
      dtype='object')


In [9]:
def encoding(df, columns):
    for i in tqdm(columns):
        # print(i)
        tmp = pd.get_dummies(df[i], prefix=i, prefix_sep='_')
        # df = df.merge(tmp, left_index=True, right_index=True).drop(columns=[i])
        df = pd.concat([df, tmp], axis=1)
        del tmp
    return df

if columns_encode:
    train = encoding(train, columns_encode)
    valid = encoding(valid, columns_encode)
    test = encoding(test, columns_encode)
    train = train[valid.columns]

100%|██████████| 6/6 [00:00<00:00, 22.80it/s]


### Normilize data

In [10]:
print(train.columns[:])
columns_norm = ['ETA', 'latitude', 'del_latitude', 'longitude', 'del_longitude',
       'EDA', 'route_steps', 'route_dist', 'route_dist_sum', 'haversine_dist',
       'manhattan_dist', 'havers_pickup_center', 'havers_dropoff_center',
       'angle', 'over_range', 'ready_time', 'collcted_time',
       'ready_collected_dif']

Index(['ETA', 'RTA', 'latitude', 'del_latitude', 'longitude', 'del_longitude',
       'EDA', 'route_steps', 'route_dist', 'route_dist_sum',
       ...
       'hour_of_week_158', 'hour_of_week_159', 'hour_of_week_160',
       'hour_of_week_161', 'hour_of_week_162', 'hour_of_week_163',
       'hour_of_week_164', 'hour_of_week_165', 'hour_of_week_166',
       'hour_of_week_167'],
      dtype='object', length=372)


In [0]:
scaleCoder = StandardScaler()
scaleCoder.fit(np.concatenate((train[columns_norm].values, valid[columns_norm].values, test[columns_norm].values)))

def transform(train, valid, test):
    train[columns_norm] = scaleCoder.transform(train[columns_norm])
    valid[columns_norm] = scaleCoder.transform(valid[columns_norm])
    test[columns_norm] = scaleCoder.transform(test[columns_norm])

if columns_norm:
    transform(train, valid, test)

## Split X and Y

In [0]:
X_train = train.drop(columns=[target_col]).values
y_train = train[target_col].values
del train
X_valid = valid.drop(columns=[target_col]).values
y_valid = valid[target_col].values
del valid
X_test = test.values
del test

## **Neural Network**

### Reshape data
Transform to 2D array

In [0]:
X_train = X_train.reshape(-1, X_train.shape[1], 1)
y_train = y_train.reshape(-1, 1)
X_valid = X_valid.reshape(-1, X_valid.shape[1], 1)
y_valid = y_valid.reshape(-1, 1)
X_test = X_test.reshape(-1, X_test.shape[1], 1)

### Create model

In [18]:
def fully_connected_1D(input_shape):
    model = Sequential()
    model.add(Flatten(input_shape=input_shape))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(12, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='relu'))
    return model

def fully_connected(input_shape):
    model = Sequential()
    model.add(Dense(64, activation='relu', input_dim=input_shape))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(12, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='relu'))
    return model

if len(X_train.shape) == 3:
    model = fully_connected_1D((X_train.shape[1], X_train.shape[2]))
elif len(X_train.shape) == 2:
    model = fully_connected((X_train.shape[1]))
X_train.shape

(824359, 371, 1)

### Train model

In [21]:
print(model.summary())
model.compile(optimizer=Adam(), 
              loss=MAPE,
              metrics=[MAE])

history = model.fit(X_train, y_train,epochs=150, batch_size=2000, verbose=2,
          callbacks=[ModelCheckpoint(PATH + "weights.h5py", monitor='val_loss',
                                     save_best_only=True, save_weights_only=True),
                     ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=5, verbose=0,
                                       mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)],
          validation_data=(X_valid, y_valid))

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 371)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 64)                23808     
_________________________________________________________________
dense_6 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_7 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_8 (Dense)              (None, 12)                204       
_________________________________________________________________
dense_9 (Dense)              (None, 8)                 104       
_________________________________________________________________
dense_10 (Dense)             (None, 1)                

In [0]:
model.load_weights(PATH + "weights.h5py")

### Score model

In [23]:
def score_model(m, ModelName, X_tr, y_tr, X_val, y_val):
    y_pred_tr = m.predict(X_tr)
    y_pred_val = m.predict(X_val)
    (mape_tr, mape_val) = (mean_absolute_percentage_error(y_tr.reshape(-1),
                                                          y_pred_tr.reshape(-1)),
                           mean_absolute_percentage_error(y_val.reshape(-1),
                                                          y_pred_val.reshape(-1)))
    (mae_tr, mae_val) = (mean_absolute_error(y_tr.reshape(-1),
                                             y_pred_tr.reshape(-1)),
                         mean_absolute_error(y_val.reshape(-1),
                                             y_pred_val.reshape(-1)))
    (mse_tr, mse_val) = (mean_squared_error(y_tr.reshape(-1),
                                            y_pred_tr.reshape(-1)),
                         mean_squared_error(y_val.reshape(-1),
                                            y_pred_val.reshape(-1)))
    models_score[ModelName] = [mape_val, mae_val, mse_val]
    return pd.DataFrame(data={'metric': ['mape', 'mae', 'mse'],
                              'train': [mape_tr, mae_tr, mse_tr],
                              'valid': [mape_val, mae_val, mse_val]})


score_model(model, "Neuro", X_train, y_train, X_valid, y_valid)

,metric,train,valid
0,mape,14.089018,13.953306
1,mae,94.658394,93.076843
2,mse,17774.737944,16444.081603


### Train on valid

In [0]:
# Freeze layers
for i,layer in enumerate(model.layers):
  print(i,layer.name,layer.trainable)
print()

# 6
for i,layer in enumerate(model.layers):
  if i < 4:
    layer.trainable = False
  print(i,layer.name,layer.trainable)

In [0]:
model.fit(X_val_, y_val_, epochs=10, batch_size=2048)

### Predict

In [0]:
model.evaluate(X_train_, y_train_)

In [0]:
prefix = 'results/'
y_test = model.predict(X_test)
test['Prediction'] = y_test.reshape(-1, 1)
test['Prediction'].to_csv(PATH + prefix + f'submit_Neuro_{models_score["Neuro"][0]}.csv')
test = test.drop(columns=['Prediction'])

## Mean submit

In [8]:
submit_names = ['submit_NeuroCNN_13.870486617088318.csv',
                'submit_NeuroCNN_13.849812746047974.csv',
                'submit_NeuroCNN_13.825468719005585.csv',
                'submit_NeuroCNN_13.498546183109283.csv',
                'submit_NeuroCNN_13.946214318275452.csv',
                'submit_NeuroCNN_13.916289806365967.csv',
                'submit_NeuroCNN_13.353970646858215.csv',
                'submit_NeuroCNN_13.070639967918396.csv',
                'submit_NeuroCNN_12.980097532272339.csv',
                'submit_NeuroCNN_13.075800240039825.csv',
                'submit_NeuroCNN_13.613928854465485.csv',
                'submit_NeuroCNN_13.977418839931488.csv',
                'submit_NeuroCNN_13.770084083080292.csv']

final_predict_df = None
prefix = 'results/'

for i in submit_names:
    if final_predict_df is None:
        final_predict_df = pd.read_csv(PATH + prefix + i, index_col=0)
    else:
        tmp = pd.read_csv(PATH + prefix + i, index_col=0)
        final_predict_df = pd.concat([final_predict_df, tmp], axis=1)
        del tmp

final_predict_df

,Prediction,Prediction,Prediction,Prediction,Prediction,Prediction,Prediction,Prediction,Prediction,Prediction,Prediction,Prediction,Prediction
Id,,,,,,,,,,,,,
0,918.95105,898.12970,904.21580,896.78424,912.91360,914.15405,906.00323,903.42770,898.61760,890.18070,884.34076,884.22577,867.84595
1,693.80760,642.28204,644.31537,683.58270,692.35754,657.22614,668.27660,667.57916,654.93680,648.05316,668.62994,714.77420,677.65010
2,344.84980,334.37340,336.93298,369.11337,357.95640,364.62310,367.31046,369.54770,356.24072,351.97836,362.90274,355.73074,335.72340
3,777.34070,763.72600,766.92126,731.13900,738.25660,759.84900,750.83435,740.88525,728.48740,727.89250,740.77924,758.91470,733.69775
4,1148.70810,1150.36600,1154.68250,1194.90200,1249.41480,1215.06520,1235.36110,1207.42930,1211.86570,1215.33100,1254.68630,1113.69820,1157.10060
...,...,...,...,...,...,...,...,...,...,...,...,...,...
89933,323.89395,311.98850,313.47570,295.18295,312.62766,301.97772,314.20782,315.03076,313.54440,312.02426,291.98180,298.48874,291.84253
89934,253.17200,266.88297,267.92984,275.25775,267.77426,266.45440,279.57920,269.35070,272.56372,270.16675,270.00833,250.01160,237.16718
89935,1166.35360,1174.23130,1178.30770,1242.13930,1188.70460,1183.08580,1252.35410,1273.83760,1273.72170,1262.20450,1215.71420,1198.01770,1237.30370


In [0]:
final_predict_df = final_predict_df.mean(axis=1)
final_predict_df.to_frame(name="Prediction").to_csv(PATH + prefix + f'submit_Neuro_MEAN.csv')